<a href="https://colab.research.google.com/github/Anacatlisboa23/Application-of-Machine-Learning-Methodologies-in-the-Search-for-New-Peptides-with-Antimicrobial-Pote/blob/main/ProtBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ProtBert Embeddings



imports:

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertModel, BertTokenizer
import torch
from sklearn.decomposition import PCA


# Prot_Bert:

In [ ]:
df_subset=pd.read_csv("/content/feaurescertas_mic (1).csv")

In [ ]:
#Continue with only the relevant columns


df_subset = df_subset.loc[:, [ 'Sequence','MIC_processado',"Gram","multiresistant","Name","Target_Organism"]]


print(df_subset)

                                               Sequence  MIC_processado  \
0               TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA            0.50   
1                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.76   
2                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.38   
3                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.76   
4     NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...            3.20   
...                                                 ...             ...   
1493                     QLEARFEPKQRNFRKRELDFEKLFANMPDY           20.00   
1494                            GLFVGLAKVAAHNNPAIAEHFQA           50.00   
1495                                            AFALKKK           64.00   
1496                                            AFALKKK           64.00   
1497                                            KKKLAYA           64.00   

               Gram  multiresistant  \
0     Gram-positivo               0   
1     Gram-negativo  

In [ ]:
# Load the tokeniser and the BERT template for proteins
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert")
model = BertModel.from_pretrained("Rostlab/prot_bert")

# Function to convert a peptide sequence into embeddings
def embed_peptide_sequence(sequence):
    # Add spaces between amino acids
    sequence = ' '.join(sequence)

    # Tokenises the peptide sequence
    tokens = tokenizer.encode(sequence, add_special_tokens=True)

    # Converts tokens to PyTorch tensors
    input_ids = torch.tensor(tokens).unsqueeze(0)  # adiciona uma dimensão para o batch

    # Obtains the embeddings of the hidden layers of the BERT model
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs.last_hidden_state  # embeddings of the last hidden layer

    # Returns the average embeddings of all the tokens in the sequence
    mean_embeddings = torch.mean(embeddings, dim=1).squeeze().numpy()

    return mean_embeddings


# Applies the embedding function to each sequence in the DataFrame and stores the resulting embeddings
#df_subset['embeddings'] = df_subset['Sequence'].apply(embed_peptide_sequence)
df_subset['embeddings'] = df_subset['Sequence'].apply(embed_peptide_sequence)
#dados['embeddings'] = dados['Sequence'].apply(embed_peptide_sequence)

# Extracts the embeddings into a separate matrix
embeddings_matrix = np.vstack(df_subset['embeddings'].values)
#embeddings_matrix = np.vstack(dados['embeddings'].values)

# Reduce the dimensionality of embeddings using PCA
pca = PCA(n_components=50)
reduced_embeddings = pca.fit_transform(embeddings_matrix)

# Convert the reduced embeddings into a DataFrame
reduced_embeddings_df = pd.DataFrame(reduced_embeddings, columns=[f'PC_{i+1}' for i in range(reduced_embeddings.shape[1])])

# Concatenates the reduced embedding columns to the original DataFrame

df = pd.concat([df_subset.drop(columns=['embeddings']), reduced_embeddings_df], axis=1)
#df = pd.concat([dados.drop(columns=['embeddings']), reduced_embeddings_df], axis=1)


print(df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

                                               Sequence  MIC_processado  \
0               TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA            0.50   
1                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.76   
2                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.38   
3                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT            0.76   
4     NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...            3.20   
...                                                 ...             ...   
1493                     QLEARFEPKQRNFRKRELDFEKLFANMPDY           20.00   
1494                            GLFVGLAKVAAHNNPAIAEHFQA           50.00   
1495                                            AFALKKK           64.00   
1496                                            AFALKKK           64.00   
1497                                            KKKLAYA           64.00   

               Gram  multiresistant  \
0     Gram-positivo               0   
1     Gram-negativo  

In [ ]:
df.to_csv("/content/CERTOO_PROTBERT_MIC.csv",index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download('/content/PROTBERT_part1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#df_merged.to_csv("/content/PROTBERT_inteiro.csv", index=False)